In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Q1. 1. Who are the top 5 users in Moscow with the highest number of followers? List their login in order, comma-separated.

In [2]:
import pandas as pd

# Load the users.csv data
users_df = pd.read_csv('users.csv')
top5 = users_df.sort_values(by='followers', ascending=False).head()
print(','.join(top5['login'].tolist()))



AlexGyver,carlcastanas,sergeyshaykhullin,alexey-goloburdin,richardroberti


In [3]:

users_df.head()
users_df['hireable'].value_counts()


,count
hireable,
True,134


Q2. 2. Who are the 5 earliest registered GitHub users in Moscow? List their login in ascending order of created_at, comma-separated.

In [4]:
import pandas as pd

# Load the users.csv data
users_df = pd.read_csv('users.csv')

# Convert created_at to datetime format for accurate sorting
users_df['created_at'] = pd.to_datetime(users_df['created_at'])

# Sort users by created_at in ascending order and select the top 5 earliest users
earliest_users = users_df.sort_values(by='created_at', ascending=True).head(5)

# Get the 'login' values for the earliest users and join them in a comma-separated format
earliest_user_logins = ','.join(earliest_users['login'].tolist())

print("5 earliest registered users in Moscow:", earliest_user_logins)


5 earliest registered users in Moscow: maxlapshin,veged,alexeyr,alec-c4,alno


Q3. What are the 3 most popular license among these users? Ignore missing licenses. List the license_name in order, comma-separated.

In [5]:
import pandas as pd

# Load the repositories.csv data
repos_df = pd.read_csv('repositories.csv')

# Filter out rows where 'license_name' is missing or null
repos_with_licenses = repos_df.dropna(subset=['license_name'])

# Count occurrences of each license_name and get the top 3 most common licenses
top_licenses = repos_with_licenses['license_name'].value_counts().head(3)

# Join the top license names as a comma-separated string
top_license_names = ','.join(top_licenses.index.tolist())

print("3 most popular licenses:", top_license_names)


3 most popular licenses: mit,other,apache-2.0


Q4. Which company do the majority of these developers work at?
Company (cleaned up as explained above)

In [6]:
import pandas as pd

# Load the users.csv data
users_df= pd.read_csv('users.csv')

# Clean up the company names
def clean_company_name(company):
    if pd.isna(company):  # Check for missing values
        return ""
    company = company.strip().upper()  # Trim whitespace and convert to uppercase
    if company.startswith('@'):
        company = company[1:]  # Remove leading '@' symbol if present
    return company

# Apply cleaning function to the 'company' column
users_df['company'] = users_df['company'].apply(clean_company_name)

# Filter out empty company values
companies_with_names = users_df[users_df['company'] != ""]

# Find the most common company
most_common_company = companies_with_names['company'].value_counts().idxmax()

print("The company where the majority of developers work:", most_common_company)


The company where the majority of developers work: YANDEX


Q5. Which programming language is most popular among these users?

In [7]:
import pandas as pd

# Load the repositories.csv data
repos_df = pd.read_csv('repositories.csv')

# Filter out rows where 'language' is missing
repos_with_languages = repos_df.dropna(subset=['language'])

# Count occurrences of each language and find the most common one
most_common_language = repos_with_languages['language'].value_counts().idxmax()

print("The most popular programming language:", most_common_language)
repos_df['language'].value_counts().head(1)


The most popular programming language: JavaScript


,count
language,
JavaScript,2972


Q6. Which programming language is the second most popular among users who joined after 2020?

In [8]:
users_df = pd.read_csv('users.csv')
users_df.head()
users_df['hireable'] = users_df['hireable'].fillna(False).astype(bool)
users_df['created_at'] = pd.to_datetime(users_df['created_at'])

# Filter users who joined after 2020

users_after_2020 = users_df[users_df['created_at'] > '2020-01-01']
users_after_2020.head()
repos_2020 = repos_df[repos_df['login'].isin(users_after_2020['login'].tolist())]
repos_2020['language'].value_counts().head()



<ipython-input-8-b549b915825b>:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  users_df['hireable'] = users_df['hireable'].fillna(False).astype(bool)


,count
language,
JavaScript,227
Python,162
CSS,116
HTML,114
TypeScript,87


Q7. Which language has the highest average number of stars per repository?

In [9]:
import pandas as pd

# Load the repositories.csv data
repos_df = pd.read_csv('repositories.csv')

# Filter out rows with missing 'language' or 'stargazers_count'
repos_with_stars = repos_df.dropna(subset=['language', 'stargazers_count'])

# Group by 'language' and calculate the average stars
language_avg_stars = repos_with_stars.groupby('language')['stargazers_count'].mean()

# Find the language with the highest average stars
top_language = language_avg_stars.idxmax()
top_language_avg_stars = language_avg_stars.max()

print(f"The language with the highest average stars per repository is: {top_language} with an average of {top_language_avg_stars:.2f} stars.")


The language with the highest average stars per repository is: Pascal with an average of 486.76 stars.


Q8. Let's define leader_strength as followers / (1 + following). Who are the top 5 in terms of leader_strength? List their login in order, comma-separated.

In [108]:
import pandas as pd

# Load the users.csv data
users_df = pd.read_csv('users.csv')

# Calculate leader_strength as followers / (1 + following)
users_df['leader_strength'] = users_df['followers'] / (1 + users_df['following'])

# Sort users by leader_strength in descending order
top_leaders = users_df.sort_values(by='leader_strength', ascending=False).head(5)

# Get the top 5 logins in order
top_leader_logins = ",".join(top_leaders['login'].tolist())

print("The top 5 users in terms of leader_strength are:",top_leader_logins)


The top 5 users in terms of leader_strength are: AlexGyver,alexey-goloburdin,yandex,yandexdataschool,esokolov


Q9. What is the correlation between the number of followers and the number of public repositories among users in Moscow?

In [11]:
correlation = users_df['followers'].corr(users_df['public_repos'])
correlation

0.05187844158212267

Q10. Does creating more repos help users get more followers? Using regression, estimate how many additional followers a user gets per additional public repository.

In [109]:
import csv
import numpy as np
followers = []
public_repos = []
with open('users.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        followers_count = int(row['followers'])
        public_repos_count = int(row['public_repos'])
        followers.append(followers_count)
        public_repos.append(public_repos_count)
if len(followers) > 1 and len(public_repos) > 1:
    slope, intercept = np.polyfit(public_repos, followers, 1)

    print(f"{slope:.3f}")
else:
    print("Error")

0.209


Q11. Do people typically enable projects and wikis together? What is the correlation between a repo having projects enabled and having wiki enabled?

In [110]:
if repos_df['has_projects'].dtype == 'object':
    repos_df['has_projects'] = repos_df['has_projects'].map({'true': True, 'false': False})
if repos_df['has_wiki'].dtype == 'object':
    repos_df['has_wiki'] = repos_df['has_wiki'].map({'true': True, 'false': False})

correlation = repos_df['has_projects'].corr(repos_df['has_wiki'])

print(round(correlation, 3))

0.345


Q12. Do hireable users follow more people than those who are not hireable?

In [111]:
users_df=pd.read_csv('users.csv')
users_df.head()
user_hireables=users_df[users_df['hireable']=='true']
user_not_hireables=users_df[users_df['hireable']=='none']
hireable_avg_following = user_hireables['following'].mean()
non_hireable_avg_following = user_not_hireables['following'].mean()
difference = hireable_avg_following - non_hireable_avg_following
difference





-29.402525832376575

13. Some developers write long bios. Does that help them get more followers? What's the correlation of the length of their bio (in Unicode characters) with followers? (Ignore people without bios)
Regression slope of followers on bio word count (to 3 decimal places, e.g. 12.345 or -12.345

In [124]:
import pandas as pd
import numpy as np

# Load the users.csv data
users_df = pd.read_csv('users.csv')

# Remove rows with missing bios
users_df = users_df.dropna(subset=['bio'])
word_Count=[]
for bio in users_df['bio']:
    word_Count.append(len(bio))
users_df['bio_length'] = word_Count
# Calculate the length of each bio in Unicode characters


# Calculate the regression slope of followers on bio length
slope, intercept = np.polyfit(users_df['bio_length'], users_df['followers'], 1)

print(f"{slope:}")

0.11939805546965979


In [125]:
from sklearn.linear_model import LinearRegression
users=pd.read_csv('users.csv')
users_with_bio = users[(users['bio'].notna()) & (users['bio'] != '')].copy()
users_with_bio.loc[:, 'bio_len'] = users_with_bio['bio'].str.len()

X = users_with_bio['bio_len'].values.reshape(-1,1)
y = users_with_bio['followers']

lr2 = LinearRegression()
lr2.fit(X, y)
lr2.coef_[0]

0.1193980554696631

Q14. Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated

In [84]:
import csv
from collections import Counter
from datetime import datetime

weekend_repo_counts = Counter()

with open('repositories.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)

    for row in reader:
        created_at = row.get('created_at', '')
        if created_at:
            created_date = datetime.fromisoformat(created_at[:-1])

            if created_date.weekday() in [5, 6]:
                user_login = row['login']
                weekend_repo_counts[user_login] += 1

top_users = weekend_repo_counts.most_common(5)

top_logins = [user[0] for user in top_users]

print(','.join(top_logins))

VN0,kuggaa,reverse-ex,Mirocow,vslinko


Q15. Do people who are hireable share their email addresses more often?
[fraction of users with email when hireable=true] minus [fraction of users with email for the rest] (to 3 decimal places, e.g. 0.123 or -0.123)

In [113]:
users_df.head()
a=users_df[users_df['hireable']=='true'].shape[0]
b=users_df[users_df['hireable']=='none'].shape[0]
hireable_with_email = users_df[(users_df['hireable'] =='true') & (users_df['email'].notna())].shape[0]
not_hireable= users_df[(users_df['hireable'] == 'none') & (users_df['email'].notna())].shape[0]
hireable_with_email_fraction = hireable_with_email / a
not_hireable_fraction = not_hireable / b
email_difference = round(hireable_with_email_fraction - not_hireable_fraction, 3)
email_difference


0.189

Q16. Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.) What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)
Number of users with the most common surname

In [34]:
new_users = users_df[users_df['name'].notna()].copy()
new_users['surname'] = new_users['name'].str.split().str[-1].str.strip()
surname_counts = new_users['surname'].value_counts()
max_count = surname_counts.max()
common_surnames = surname_counts[surname_counts == max_count].index.tolist()
common_surnames.sort()
print(','.join(common_surnames))

Romanov
